In [1]:
### DO NOT RUN THIS.  It builds the BkAvgAst... tables, which are supplied in the processing tables zip file.
### 
### NOTE:  Major change due to conversion of some bank holding companies back to bank structures.
###        (This removes the Parent (NAMEHCR) and ParRSSD (RSSDHCR) fields from the institutions2.zip listing.)
###        

import numpy as np
import pandas as pd
from datetime import datetime
#import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.chdir('C:/BankRisk')


In [2]:
rk = pd.read_csv("BkAvgAstPR_FailDate_201909.csv", index_col=0)
rk['xAsOf'] = pd.to_datetime(rk['xAsOf'])
rk['nAsOf'] = pd.to_datetime(rk['nAsOf'])
rk['FailDate'] = pd.to_datetime(rk['FailDate'])
rk.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2019-09-30  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2019-09-30      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2019-09-30           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2019-09-30     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.516249e+09  1.491699e+09  5.364465e+08  7.085924e+08   
1 2001-03-31  1.290971e+09  1.294099e+09  6.762368e+08  8.324937e+08   
2 2001-03-31  1.078882e+09  1.075874e+09  5.141358e+08  2.955661e+08   
3 2001-03-31  9.246635e+08  9.130482e+08  5.535713e+08  6.474181e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate    pr_eTA   pr_eTAcs    pr_eTL  \
0  2.556333e+08  9.642257e+08        NaT  9.230096   9.230096  5.726881   
1  1.045938e+08  9.370875e+08 2009-01-16  7.858724  17.088820  7.219225   
2  4.454573e+08  7.410233e+08 2008-11-23  6.567645  23.656465  5.488702   
3  6.676371e+07  7.141819e+08        NaT  5.628846  29.285311  5.909699   
4  2.469325e+07  2.951088e+08        NaT  2.688502  31.973813  2.718399   

    pr_eTLcs    pr_eDD   pr_eDDcs  
0   5.726881  7.564640   7.564640  
1  12.946106  8.887359  16.452000  
2  18.434807  3.155341  19.607341  
3  24.344506  6.911569  26.518910  
4  27.062905  2.886845  29.405755

In [3]:
len(rk) # 10787

10787

In [4]:
bhc = pd.read_csv("INSTITUTIONS2_201911.csv", encoding='latin-1') # Warning: This is latest available file as of 2019-April.
bhc.head()

STNAME  CERT  DOCKET  ACTIVE              ADDRESS      ASSET BKCLASS  \
0  Connecticut     4       0       0       22 Main Street     48,570      NM   
1  Connecticut     6       0       0  81 West Main Street    624,655      NM   
2        Maine     8       0       0      One City Center  1,699,404      SM   
3        Maine     9    9781       0       66 Main Street    539,169      SM   
4        Maine    10       0       0  8 Washington Street     55,692      SM   

   CHANGEC1  CHANGEC2  CHANGEC3  ...  pte08n528  pte09n528  pte10n528  \
0       223         0         0  ...        NaN        NaN        NaN   
1       223       999         0  ...        NaN        NaN        NaN   
2       223         0         0  ...        NaN        NaN        NaN   
3       223         0         0  ...        NaN        NaN        NaN   
4       223         0         0  ...        NaN        NaN        NaN   

  pte01n529 pte02n529 pte03n529  pte04n529  pte05n529 pte06n529 pwebaddr  
0       NaN       NaN       NaN        NaN        NaN       NaN      NaN  
1       NaN       NaN       NaN        NaN        NaN       NaN      NaN  
2       NaN       NaN       NaN        NaN        NaN       NaN      NaN  
3       NaN       NaN       NaN        NaN        NaN       NaN      NaN  
4       NaN       NaN       NaN        NaN        NaN       NaN      NaN  

[5 rows x 151 columns]

In [5]:
bhc.replace(to_replace='orporated', value='', inplace=True, regex=True)
bhc.replace(to_replace='Irrevocable', value='Irrev', inplace=True, regex=True)
bhc.replace(to_replace='Employee Stock Ownership ', value='ESO', inplace=True, regex=True)
bhc.replace(to_replace='orporation', value='orp', inplace=True, regex=True)
bhc.replace(to_replace='inancial', value='in', inplace=True, regex=True)
bhc.replace(to_replace='Holding Company', value='HC', inplace=True, regex=True)
bhc.replace(to_replace=', Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace=' Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace=', The', value='', inplace=True, regex=True)

In [6]:
len(bhc) # 27617

27617

In [7]:
bhc.head(2)

STNAME  CERT  DOCKET  ACTIVE              ADDRESS    ASSET BKCLASS  \
0  Connecticut     4       0       0       22 Main Street   48,570      NM   
1  Connecticut     6       0       0  81 West Main Street  624,655      NM   

   CHANGEC1  CHANGEC2  CHANGEC3  ...  pte08n528  pte09n528  pte10n528  \
0       223         0         0  ...        NaN        NaN        NaN   
1       223       999         0  ...        NaN        NaN        NaN   

  pte01n529 pte02n529 pte03n529  pte04n529  pte05n529 pte06n529 pwebaddr  
0       NaN       NaN       NaN        NaN        NaN       NaN      NaN  
1       NaN       NaN       NaN        NaN        NaN       NaN      NaN  

[2 rows x 151 columns]

In [8]:
bhc2=bhc[['CERT','FED_RSSD','NAME','STALP','BKCLASS','OFFICES','NAMEHCR','RSSDHCR','STALPHCR','ULTCERT']]
#bhc2.columns = [['Cert','IDRSSD','Name','ST','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']]
bhc2.rename(index=str, columns={"CERT":"Cert", "FED_RSSD":"IDRSSD", "NAME":"Name", "STALP":"ST", "BKCLASS":"BkClass", \
            "OFFICES":"Offices", "NAMEHCR":"Parent", "RSSDHCR":"ParRSSD", "STALPHCR":"ParST", "ULTCERT":"UltCert"}, inplace=True)
len(bhc2) # 27617

27617

In [9]:
bhc2.head(2)

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   

  Parent  ParRSSD ParST  UltCert  
0    NaN      NaN   NaN     3510  
1    NaN      NaN   NaN     3510

In [10]:
bhc2.columns

Index(['Cert', 'IDRSSD', 'Name', 'ST', 'BkClass', 'Offices', 'Parent',
       'ParRSSD', 'ParST', 'UltCert'],
      dtype='object')

In [11]:
bhc2[bhc2['Parent']>' '][0:10]

Cert  IDRSSD                                 Name  ST BkClass  Offices  \
5     14   35301  State Street Bank and Trust Company  MA      SM      3.0   
15    35  749635                           AuburnBank  AL      SM      8.0   
19    39  430036            Robertson Banking Company  AL      NM      7.0   
21    41  432432                   Phenix-Girard Bank  AL      NM      4.0   
25    46  113937            Merchants Bank of Alabama  AL      NM      5.0   
26    47  390336               Traders & Farmers Bank  AL      NM      7.0   
29    50  470032                Community Spirit Bank  AL      NM      5.0   
30    51  382537                   The Bank of Vernon  AL      NM      3.0   
31    52  293437        The Citizens Bank of Winfield  AL      NM      1.0   
33    54  672537         The Exchange Bank of Alabama  AL      NM      5.0   

                               Parent    ParRSSD ParST  UltCert  
5                   State Street Corp  1111435.0    MA       14  
15        Auburn National Bancorp Inc  1129533.0    AL       35  
19                            Rbc Inc  2526755.0    AL       39  
21       Phenix-Girard Bancshares Inc  2013246.0    AL       41  
25         Merchants Fin Services Inc  1082955.0    AL       46  
26   Traders & Farmers Bancshares Inc  3467305.0    AL       47  
29  Independent Bancshares IncESOPlan  3587294.0    AL       50  
30        First Vernon Bancshares Inc  1139998.0    AL       51  
31   Citizens Bancorp Of Winfield Inc  1833616.0    AL       52  
33                       Gadsden Corp  1079067.0    AL       54

In [14]:
#bhc2[(bhc2['ParRSSD']>0)&pd.isnull(bhc2['Parent'])]  ## should be null list

In [15]:
#bhc2[(bhc2['UltCert']>0)&pd.isnull(bhc2['Parent'])].head()

In [16]:
## extract ticker info from prior listing: (rkhn was expanded output from br001f_NewBHCtickers code)
rkh = pd.read_csv("BkAvgAstPR_FailDate_BHC2_ult201903.csv", index_col=0)

#len(rkh) # 10775
len(rkh) # 10776

10776

In [17]:
len(rkh[rkh['Ticker']>'']) # 3266

3266

In [18]:
#uex = hs3[['Date','UE','UE_'+st]].ix[hs3['UE'].idxmax()]
rkh2 = rkh[rkh['ParRSSD']>0][['Rank','ParRSSD','ParST','Ticker']]
len(rkh2) # 8794

8794

In [19]:
rkh2.tail(2)

Rank    ParRSSD ParST Ticker
10772  10773  1094314.0    MO  CBCYB
10775  10776  3447567.0    TX    NaN

In [20]:
rkh3 = rkh2.groupby(['ParRSSD','Ticker']).min()
rkh3.sort_index(by='Rank',inplace=True)
len(rkh3) # 402

402

In [21]:
rkh3.head()

Rank ParST
ParRSSD   Ticker            
1039502.0 JPM        1    NY
1073757.0 BAC        2    NC
1951350.0 C          3    NY
1120754.0 WFC        4    CA
1119794.0 USB        6    MN

In [22]:
rkh4 = rkh3.reset_index(drop=False)
rkh4.head()

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC
2  1951350.0      C     3    NY
3  1120754.0    WFC     4    CA
4  1119794.0    USB     6    MN

In [23]:
rkh5=rkh4[['ParRSSD','Ticker']].groupby('Ticker').count()
rkh5.head()

ParRSSD
Ticker         
ABCB          1
ABTX          1
ABYB          1
ACBI          1
ACNB          1

In [24]:
rkh5[rkh5['ParRSSD']>1] # finds tickers with more than one ParRSSD code:

Empty DataFrame
Columns: [ParRSSD]
Index: []

In [25]:
rkh[rkh['Ticker']=="BHWB"].iloc[:,0:12] # check on Ticker that was misidentified in earlier runs (should be 2 banks only)

Rank  IDRSSD       xAsOf           xName xState    xCert       nAsOf  \
1737  1738   27548  2019-03-31  BLACKHAWK BANK     WI  14078.0  2001-03-31   
7747  7748  388931  2003-09-30   FIRST BANK BC     IL  16551.0  2001-03-31   

     FailDate BkClass  Offices                 Parent    ParRSSD  
1737      NaN      NM      9.0  Blackhawk Bancorp Inc  1491913.0  
7747      NaN      NM      4.0  Blackhawk Bancorp Inc  1491913.0

In [26]:
bhc3 = bhc2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
bhc3.head()

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   
2     8  422406                     Fleet Bank of Maine  ME      SM     55.0   
3     9  563907                     Union Trust Company  ME      SM     13.0   
4    10  112109               Northeast Bank of Sanford  ME      SM      5.0   

  Parent  ParRSSD ParST  UltCert Ticker  
0    NaN      NaN   NaN     3510    NaN  
1    NaN      NaN   NaN     3510    NaN  
2    NaN      NaN   NaN     3510    NaN  
3    NaN      NaN   NaN     4255    NaN  
4    NaN      NaN   NaN     3510    NaN

In [27]:
bhc3[bhc3['Cert']==3510]

Cert  IDRSSD                                   Name  ST BkClass  \
2233  3510  480228  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert Ticker  
2233   4276.0  Bank Of America Corp  1073757.0    NC     3510    BAC

In [28]:
bhc2[bhc2['Cert']==3510]

Cert  IDRSSD                                   Name  ST BkClass  \
2233  3510  480228  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert  
2233   4276.0  Bank Of America Corp  1073757.0    NC     3510

In [29]:
rk2 = rk.merge(bhc2[['Cert','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']],how='left',left_on='xCert',\
              right_on='Cert')
rk2.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2019-09-30  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2019-09-30      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn  ...   pr_eTLcs    pr_eDD  \
0  1.516249e+09  1.491699e+09  5.364465e+08  ...   5.726881  7.564640   
1  1.290971e+09  1.294099e+09  6.762368e+08  ...  12.946106  8.887359   

   pr_eDDcs    Cert  BkClass  Offices                Parent    ParRSSD  ParST  \
0   7.56464   628.0        N   5040.0  Jpmorgan Chase & Co.  1039502.0     NY   
1  16.45200  3510.0        N   4276.0  Bank Of America Corp  1073757.0     NC   

   UltCert  
0    628.0  
1   3510.0  

[2 rows x 27 columns]

In [30]:
rkh4.head(2)

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC

In [31]:
rk3 = rk2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
rk3.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2019-09-30  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2019-09-30      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn  ...    pr_eDD  pr_eDDcs    Cert  \
0  1.516249e+09  1.491699e+09  5.364465e+08  ...  7.564640   7.56464   628.0   
1  1.290971e+09  1.294099e+09  6.762368e+08  ...  8.887359  16.45200  3510.0   

  BkClass  Offices                Parent    ParRSSD  ParST  UltCert  Ticker  
0       N   5040.0  Jpmorgan Chase & Co.  1039502.0     NY    628.0     JPM  
1       N   4276.0  Bank Of America Corp  1073757.0     NC   3510.0     BAC  

[2 rows x 28 columns]

In [32]:
rk3c = list(rk3.columns)
rk3c2 = rk3c[0:7] + ['FailDate'] + rk3c[-7:] + rk3c[7:13] + rk3c[14:20]
rk3c, rk3c2

(['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'FailDate',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs',
  'Cert',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker'],
 ['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'FailDate',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs'])

In [33]:
rk4 = rk3[rk3c2]
rk4.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2019-09-30  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2019-09-30      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

    FailDate BkClass  Offices  ...       aeTotLn        aeDDep        aeFDep  \
0        NaT       N   5040.0  ...  5.364465e+08  7.085924e+08  2.556333e+08   
1 2009-01-16       N   4276.0  ...  6.762368e+08  8.324937e+08  1.045938e+08   

       aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs    pr_eDD  pr_eDDcs  
0  9.642257e+08  9.230096   9.230096  5.726881   5.726881  7.564640   7.56464  
1  9.370875e+08  7.858724  17.088820  7.219225  12.946106  8.887359  16.45200  

[2 rows x 27 columns]

In [34]:
## New code (May 2019) to add holding company values for Parent, ParRSSD, Ticker for Zions Bancorp (since this data)
## was recently removed due to legal changes to terminate Fed CCAR oversight
## If other companies do the same, their data will be added back here
## (This is essential for the historical CCAR 31-bank group to compute properly)

In [35]:
rk4.loc[rk4.xCert.isin([2270]),['IDRSSD','xName','xCert','Ticker','Parent','ParRSSD']]

IDRSSD                     xName   xCert Ticker Parent  ParRSSD
78  276579  ZIONS BANCORPORATION, NA  2270.0    NaN    NaN      0.0

In [36]:
rk4.loc[rk4.xCert == 2270, 'Parent'] = 'ZION'
rk4.loc[rk4.xCert == 2270, 'Ticker'] = 'ZION'
rk4.loc[rk4.xCert == 2270, 'ParRSSD'] = 1027004.0

In [37]:
rk4.loc[rk4.xCert.isin([2270]),['IDRSSD','xName','xCert','Ticker','Parent','ParRSSD']]

IDRSSD                     xName   xCert Ticker Parent    ParRSSD
78  276579  ZIONS BANCORPORATION, NA  2270.0   ZION   ZION  1027004.0

In [38]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC_nohist201909.csv")

In [39]:
len(rk4) # 10787

10787

In [40]:
## find bhcs with totast above 700MM and no ticker:
rkm = rk4[(rk4['ParRSSD']>0)&(rk4['aeTotAst']>700000)&pd.isnull(rk4['Ticker'])]
len(rkm) # 259

259

In [41]:
rkm.head()

Rank   IDRSSD      xAsOf                      xName xState    xCert  \
25    26  3150447 2019-09-30        CHARLES SCHWAB BANK     NV  57450.0   
33    34   619877 2019-09-30  USAA FEDERAL SAVINGS BANK     TX  32188.0   
50    51  1216022 2019-09-30             SYNCHRONY BANK     UT  27314.0   
55    56   370271 2019-09-30               E*TRADE BANK     VA  30746.0   
77    78  2735146 2019-09-30                  TIAA, FSB     FL  34775.0   

        nAsOf FailDate BkClass  Offices  ...       aeTotLn        aeDDep  \
25 2003-06-30      NaT      SA      1.0  ...  9.529729e+06  8.648340e+07   
33 2012-03-31      NaT      SA      1.0  ...  4.415471e+07  6.429210e+07   
50 2001-03-31      NaT      SA      4.0  ...  3.987647e+07  3.458479e+07   
55 2012-03-31      NaT      SA      1.0  ...  8.848700e+06  3.618879e+07   
77 2012-03-31      NaT      SA     14.0  ...  2.212307e+07  1.815206e+07   

       aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs  \
25   0.000000  8.648340e+07  0.573487  51.298759  0.101735  43.861565   
33   0.000000  6.429210e+07  0.440049  55.107032  0.471377  48.244104   
50  31.589744  3.458482e+07  0.288529  61.252882  0.425705  55.602555   
55   0.000000  3.618879e+07  0.268698  62.614445  0.094465  56.874444   
77   0.000000  1.815206e+07  0.159361  67.024280  0.236177  62.316062   

      pr_eDD   pr_eDDcs  
25  0.923261  49.094175  
33  0.686356  53.361314  
50  0.369213  60.176137  
55  0.386337  61.721016  
77  0.193784  66.478224  

[5 rows x 27 columns]

In [42]:
rkm.to_csv('rk_missingtickers201909.csv')

In [43]:
##############################################################################

In [44]:
## start up here after filling in missing tickers

In [45]:
## filled in many missing tickers

In [46]:
rkm2 = pd.read_csv('rk_missingtickers201909a.csv',index_col=0)
rkm2.head()

Rank   IDRSSD      xAsOf                      xName xState  xCert  \
25    26  3150447  9/30/2019        CHARLES SCHWAB BANK     NV  57450   
33    34   619877  9/30/2019  USAA FEDERAL SAVINGS BANK     TX  32188   
50    51  1216022  9/30/2019             SYNCHRONY BANK     UT  27314   
55    56   370271  9/30/2019               E*TRADE BANK     VA  30746   
77    78  2735146  9/30/2019                  TIAA, FSB     FL  34775   

        nAsOf FailDate BkClass  Offices  ...       aeTotLn       aeDDep  \
25  6/30/2003      NaN      SA        1  ...  9.529729e+06  86483400.43   
33  3/31/2012      NaN      SA        1  ...  4.415471e+07  64292095.48   
50  3/31/2001      NaN      SA        4  ...  3.987647e+07  34584793.21   
55  3/31/2012      NaN      SA        1  ...  8.848700e+06  36188787.94   
77  3/31/2012      NaN      SA       14  ...  2.212307e+07  18152059.29   

       aeFDep     aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs  \
25   0.000000  86483400.43  0.573487  51.298759  0.101735  43.861565   
33   0.000000  64292095.48  0.440049  55.107032  0.471377  48.244104   
50  31.589744  34584824.79  0.288529  61.252882  0.425705  55.602555   
55   0.000000  36188787.94  0.268698  62.614445  0.094465  56.874444   
77   0.000000  18152059.29  0.159361  67.024280  0.236177  62.316062   

      pr_eDD   pr_eDDcs  
25  0.923261  49.094175  
33  0.686356  53.361314  
50  0.369213  60.176137  
55  0.386337  61.721016  
77  0.193784  66.478224  

[5 rows x 27 columns]

In [47]:
rkm3 = rkm2[rkm2['Ticker']>' '][['ParRSSD','Ticker']]
rkm3.head()

ParRSSD Ticker
25   1026632   SCHW
50   4504654    SYF
55   3412583   ETFC
79   1232497     RY
123  1025608    FHB

In [48]:
rkm3p = set(rkm3['ParRSSD'])
len(rkm3), len(rkm3p) # 39, 35

(39, 35)

In [49]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','Ticker']]

xName                             Parent  \
25                     CHARLES SCHWAB BANK                Charles Schwab Corp   
50                          SYNCHRONY BANK                      Synchrony Fin   
55                            E*TRADE BANK                   E*trade Fin Corp   
79                      CITY NATIONAL BANK               Royal Bank Of Canada   
123                    FIRST HAWAIIAN BANK                 First Hawaiian Inc   
126                     FLAGSTAR BANK, FSB                    Mp (Thrift) Llc   
171           CAPITOL FEDERAL SAVINGS BANK            Capitol Federal Fin Inc   
207                             FIRST BANK                            Fb Corp   
224                              AXOS BANK                       Axos Fin Inc   
233   WILMINGTON SAVINGS FUND SOCIETY, FSB                      Wsfs Fin Corp   
242          AMERICAN SAVINGS BANK, F.S.B.   Hawaiian Electric Industries Inc   
245                       AMERANT BANK, NA                Amerant Bancorp Inc   
259                     PLAINSCAPITAL BANK  Diamond Hth Stock Company Gp, Llc   
266                    DIME COMMUNITY BANK      Dime Community Bancshares Inc   
269          CITY NATIONAL BANK OF FLORIDA            Empresas Juan Yarur Spa   
286                           TRUSTCO BANK               Trustco Bank Corp Ny   
304                            KEARNY BANK                    Kearny Fin Corp   
329                      MIZUHO BANK (USA)               Mizuho Fin Group Inc   
358                               METABANK                 Meta Fin Group Inc   
388            CHARLES SCHWAB PREMIER BANK                Charles Schwab Corp   
428           JOHN DEERE FINANCIAL, F.S.B.                Deere & Company Inc   
450                 RBC BANK (GEORGIA), NA               Royal Bank Of Canada   
452      FIRST FEDERAL BANK OF THE MIDWEST            First Defiance Fin Corp   
460                    ETRADE SAVINGS BANK                   E*trade Fin Corp   
461                        NORTHFIELD BANK             Northfield Bancorp Inc   
497                           ORITANI BANK                   Oritani Fin Corp   
528         PARKWAY BANK AND TRUST COMPANY                Parkway Bancorp Inc   
589               TERRITORIAL SAVINGS BANK            Territorial Bancorp Inc   
619              CHARLES SCHWAB TRUST BANK                Charles Schwab Corp   
628       NORTH AMERICAN SAVINGS BANK, FSB                       Nasb Fin Inc   
635                        WATERSTONE BANK                 Waterstone Fin Inc   
730                         WESTFIELD BANK    Western New England Bancorp Inc   
751                        CROSSFIRST BANK          Crossfirst Bankshares Inc   
780             FIRST STATE COMMUNITY BANK         First State Bancshares Inc   
813         PROVIDENT SAVINGS BANK, F.S.B.         Provident Fin Holdings Inc   
881                     WOORI AMERICA BANK                Woori Fin Group Inc   
950          SEASIDE NATIONAL BANK & TRUST           Three Shores Bancorp Inc   
959                       MALAGA BANK, FSB                    Malaga Fin Corp   
978               RIVERVIEW COMMUNITY BANK              Riverview Bancorp Inc   
2689                   FIRST GULF BANK, NA               Royal Bank Of Canada   

     Ticker  
25      NaN  
50      NaN  
55      NaN  
79      NaN  
123     NaN  
126     NaN  
171     NaN  
207     NaN  
224     NaN  
233     NaN  
242     NaN  
245     NaN  
259     NaN  
266     NaN  
269     NaN  
286     NaN  
304     NaN  
329     NaN  
358     NaN  
388     NaN  
428     NaN  
450     NaN  
452     NaN  
460     NaN  
461     NaN  
497     NaN  
528     NaN  
589     NaN  
619     NaN  
628     NaN  
635     NaN  
730     NaN  
751     NaN  
780     NaN  
813     NaN  
881     NaN  
950     NaN  
959     NaN  
978     NaN  
2689    NaN

In [50]:
rk4c = rk4.copy()

In [51]:
tdct = dict(zip(rkm3['ParRSSD'],rkm3['Ticker']))
tdct

{1026632: 'SCHW',
 4504654: 'SYF',
 3412583: 'ETFC',
 1232497: 'RY',
 1025608: 'FHB',
 4224000: 'FBC',
 4226910: 'CFFN',
 1118797: 'FRBA',
 3814310: 'AX',
 3844269: 'WSFS',
 3842957: 'HE',
 1135972: 'AMTB',
 5213690: 'HTH',
 2487650: 'DCOM',
 2833891: 'BCI',
 1048513: 'TRST',
 3099443: 'KRNY',
 3185355: 'MFG',
 2390013: 'CASH',
 1277313: 'DE',
 3316917: 'FDEF',
 3132863: 'NFBK',
 2692892: 'ORIT',
 1200692: 'PFED',
 3954681: 'TBNK',
 3841941: 'NASB',
 4523431: 'WSBF',
 3866382: 'WNEB',
 3903661: 'CFB',
 1099917: 'FSTB',
 3632493: 'PROV',
 5309306: 'WF',
 3934562: 'TSHR',
 3842528: 'MLGF',
 3180060: 'RVSB'}

In [52]:
## update missing Tickers if it exists:
for i in np.arange(len(rk4)):
  rs = rk4.ix[i]['ParRSSD']
  try:
    rk4.loc[i,'Ticker'] = tdct[rs]
  except:
    pass

In [53]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','ParRSSD','Ticker']]

xName                             Parent  \
25                     CHARLES SCHWAB BANK                Charles Schwab Corp   
50                          SYNCHRONY BANK                      Synchrony Fin   
55                            E*TRADE BANK                   E*trade Fin Corp   
79                      CITY NATIONAL BANK               Royal Bank Of Canada   
123                    FIRST HAWAIIAN BANK                 First Hawaiian Inc   
126                     FLAGSTAR BANK, FSB                    Mp (Thrift) Llc   
171           CAPITOL FEDERAL SAVINGS BANK            Capitol Federal Fin Inc   
207                             FIRST BANK                            Fb Corp   
224                              AXOS BANK                       Axos Fin Inc   
233   WILMINGTON SAVINGS FUND SOCIETY, FSB                      Wsfs Fin Corp   
242          AMERICAN SAVINGS BANK, F.S.B.   Hawaiian Electric Industries Inc   
245                       AMERANT BANK, NA                Amerant Bancorp Inc   
259                     PLAINSCAPITAL BANK  Diamond Hth Stock Company Gp, Llc   
266                    DIME COMMUNITY BANK      Dime Community Bancshares Inc   
269          CITY NATIONAL BANK OF FLORIDA            Empresas Juan Yarur Spa   
286                           TRUSTCO BANK               Trustco Bank Corp Ny   
304                            KEARNY BANK                    Kearny Fin Corp   
329                      MIZUHO BANK (USA)               Mizuho Fin Group Inc   
358                               METABANK                 Meta Fin Group Inc   
388            CHARLES SCHWAB PREMIER BANK                Charles Schwab Corp   
428           JOHN DEERE FINANCIAL, F.S.B.                Deere & Company Inc   
450                 RBC BANK (GEORGIA), NA               Royal Bank Of Canada   
452      FIRST FEDERAL BANK OF THE MIDWEST            First Defiance Fin Corp   
460                    ETRADE SAVINGS BANK                   E*trade Fin Corp   
461                        NORTHFIELD BANK             Northfield Bancorp Inc   
497                           ORITANI BANK                   Oritani Fin Corp   
528         PARKWAY BANK AND TRUST COMPANY                Parkway Bancorp Inc   
589               TERRITORIAL SAVINGS BANK            Territorial Bancorp Inc   
619              CHARLES SCHWAB TRUST BANK                Charles Schwab Corp   
628       NORTH AMERICAN SAVINGS BANK, FSB                       Nasb Fin Inc   
635                        WATERSTONE BANK                 Waterstone Fin Inc   
730                         WESTFIELD BANK    Western New England Bancorp Inc   
751                        CROSSFIRST BANK          Crossfirst Bankshares Inc   
780             FIRST STATE COMMUNITY BANK         First State Bancshares Inc   
813         PROVIDENT SAVINGS BANK, F.S.B.         Provident Fin Holdings Inc   
881                     WOORI AMERICA BANK                Woori Fin Group Inc   
950          SEASIDE NATIONAL BANK & TRUST           Three Shores Bancorp Inc   
959                       MALAGA BANK, FSB                    Malaga Fin Corp   
978               RIVERVIEW COMMUNITY BANK              Riverview Bancorp Inc   
2689                   FIRST GULF BANK, NA               Royal Bank Of Canada   

        ParRSSD Ticker  
25    1026632.0   SCHW  
50    4504654.0    SYF  
55    3412583.0   ETFC  
79    1232497.0     RY  
123   1025608.0    FHB  
126   4224000.0    FBC  
171   4226910.0   CFFN  
207   1118797.0   FRBA  
224   3814310.0     AX  
233   3844269.0   WSFS  
242   3842957.0     HE  
245   1135972.0   AMTB  
259   5213690.0    HTH  
266   2487650.0   DCOM  
269   2833891.0    BCI  
286   1048513.0   TRST  
304   3099443.0   KRNY  
329   3185355.0    MFG  
358   2390013.0   CASH  
388   1026632.0   SCHW  
428   1277313.0     DE  
450   1232497.0     RY  
452   3316917.0   FDEF  
460   3412583.0   ETFC  
461   3132863.0   NFBK  
497   2692892.0   ORIT  
528   1200692.0   PFED  
589   3954681.0

In [54]:
len(rk4[rk4['Ticker']>' ']),len(rk4c[rk4c['Ticker']>' ']) # 529, 489

(529, 489)

In [55]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC2_nohist201909.csv")

In [56]:
## process ultcert version:

In [57]:
rk5 = rk4.copy()

In [58]:
ult = set(rk5['UltCert'])
len(ult) # 5576

5576

In [59]:
bhcc = rk5[rk5['xCert'].isin(ult)][['xCert','Parent','Ticker','ParRSSD','ParST']]
bhcc.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [60]:
bhcc.dropna(subset=['Parent'],inplace=True)
len(bhcc) # 4284

4284

In [61]:
len(bhcc[bhcc['Ticker']>'']) # 524

524

In [62]:
bhcc.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [63]:
bhcc.groupby('xCert').filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

xCert                           Parent Ticker    ParRSSD ParST
38     16571.0                 Bank Of Montreal    BMO  1231333.0     0
187     4999.0                  Cadence Bancorp   CADE  4882802.0    TX
253    58564.0                Flushing Fin Corp   FFIC  2393274.0    NY
450    26342.0             Royal Bank Of Canada     RY  1232497.0     0
487    35385.0        United Community Fin Corp   UCFC  3831250.0    OH
685     4999.0                  Cadence Bancorp   CADE  4882802.0    TX
686    35525.0  Atlantic Capital Bancshares Inc   ACBI  3555686.0    GA
1019   35525.0  Atlantic Capital Bancshares Inc   ACBI  3555686.0    GA
1073    8888.0                     Ccb Fin Corp    NaN  1053496.0    MO
1202   57197.0                      Bnccorp Inc   BNCC  1248162.0    ND
1566    3116.0   Lone Star State Bancshares Inc    NaN  3641767.0    TX
1732   10973.0        Alliance Fin Services Inc    NaN  1127146.0    MN
1792      89.0   White River Bancshares Company    NaN  3306589.0    AR
1814   16571.0                 Bank Of Montreal    BMO  1231333.0     0
1926   57497.0           Pathfinder Bancorp Inc    NaN  2596776.0    NY
2207   58564.0                Flushing Fin Corp   FFIC  2393274.0    NY
2689   26342.0             Royal Bank Of Canada     RY  1232497.0     0
2783   13198.0               Cornerstone HC Inc    NaN  3435283.0    ND
4069   11310.0                  Fgh Bancorp Inc    NaN  2527024.0    IL
4218   35274.0        Blue Ridge Bankshares Inc    NaN  1426960.0    VA
4270   35385.0        United Community Fin Corp   UCFC  3831250.0    OH
4385   13198.0               Cornerstone HC Inc    NaN  3435283.0    ND
4597   17636.0  First Belleville Bancshares Inc    NaN  1060917.0    KS
4729    8252.0         Jamesmark Bancshares Inc    NaN  2816803.0    MO
5992    3116.0   Lone Star State Bancshares Inc    NaN  3641767.0    TX
6239    1756.0      Cross County Bancshares Inc    NaN  1131611.0    AR
6787   17399.0                   First Fin Corp    NaN  1067514.0    KS
7137      89.0   White River Bancshares Company    NaN  3306589.0    AR
7144   10973.0        Alliance Fin Services Inc    NaN  1127146.0    MN
7432   13381.0   Schuyler County Bancshares Inc    NaN  1100055.0    MO
7526   35274.0        Blue Ridge Bankshares Inc    NaN  1426960.0    VA
7786    8252.0         Jamesmark Bancshares Inc    NaN  2816803.0    MO
8696   29617.0         First Mutual Holding Co.    NaN  4843090.0    OH
8795   57497.0           Pathfinder Bancorp Inc    NaN  2596776.0    NY
9086   29617.0         First Mutual Holding Co.    NaN  4843090.0    OH
9136   11310.0                  Fgh Bancorp Inc    NaN  2527024.0    IL
9683    8888.0                     Ccb Fin Corp    NaN  1053496.0    MO
9807   17399.0                   First Fin Corp    NaN  1067514.0    KS
10210  13381.0   Schuyler County Bancshares Inc    NaN  1100055.0    MO
10248  17636.0  First Belleville Bancshares Inc    NaN  1060917.0    KS
10437   1756.0      Cross County Bancshares Inc    NaN  1131611.0    AR
10533  57197.0                      Bnccorp Inc   BNCC  1248162.0    ND

In [64]:
bhcc[bhcc['xCert']==17399]

xCert          Parent Ticker    ParRSSD ParST
6787  17399.0  First Fin Corp    NaN  1067514.0    KS
9807  17399.0  First Fin Corp    NaN  1067514.0    KS

In [65]:
len(bhcc) # 4284

4284

In [66]:
bhcc2 = bhcc.drop_duplicates(subset=['xCert','Parent','Ticker','ParRSSD'],keep=False)
len(bhcc2) # 4242

4242

In [67]:
bhcc2.head()

xCert                 Parent Ticker    ParRSSD ParST
0   628.0   Jpmorgan Chase & Co.    JPM  1039502.0    NY
1  3510.0   Bank Of America Corp    BAC  1073757.0    NC
2  7213.0          Citigroup Inc      C  1951350.0    NY
3  3511.0  Wells Fargo & Company    WFC  1120754.0    CA
5  6548.0           U.S. Bancorp    USB  1119794.0    MN

In [68]:
bhcc2.set_index(['xCert'],drop=True,inplace=True)
bhcc2.head()

Parent Ticker    ParRSSD ParST
xCert                                                
628.0    Jpmorgan Chase & Co.    JPM  1039502.0    NY
3510.0   Bank Of America Corp    BAC  1073757.0    NC
7213.0          Citigroup Inc      C  1951350.0    NY
3511.0  Wells Fargo & Company    WFC  1120754.0    CA
6548.0           U.S. Bancorp    USB  1119794.0    MN

In [69]:
bhcc2.groupby(level=0).filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

Empty DataFrame
Columns: [Parent, Ticker, ParRSSD, ParST]
Index: []

In [70]:
dcrt = bhcc2[['Parent','Ticker','ParRSSD','ParST']].to_dict()
dcrt['Parent'][628]

'Jpmorgan Chase & Co.'

In [71]:
##### LOOK AT THIS!!! IMPORTANT CONDITIONAL PROCESSING FOR MISSING VALUES!!!!  ## improved 5/9/2016
##########################################################################
j = 0
rk5c = rk5.copy()
for i in np.arange(len(rk5c)):
  if (pd.isnull(rk5c.ix[i]['Parent']))|(rk5c.ix[i]['Parent']==' '):
    xc = rk5c.ix[i]['UltCert']
    try:
      rk5c.loc[i,'Parent'] = dcrt['Parent'][xc]
      rk5c.loc[i,'Ticker'] = dcrt['Ticker'][xc]
      rk5c.loc[i,'ParRSSD'] = dcrt['ParRSSD'][xc]
      rk5c.loc[i,'ParST'] = dcrt['ParST'][xc]
      j = j+1
    except:
      pass

In [72]:
len(rk5c[rk5c['Ticker']>'']) # 3430

3430

In [73]:
j # 4826

4826

In [74]:
len(rk5c) # 10787

10787

In [75]:
rk5c.to_csv("BkAvgAstPR_FailDate_BHC2_ult201909.csv")

In [76]:
# I use ZION to show how many historic banks and mergers got collapsed into one bank charter

In [77]:
rk4.loc[rk4.Ticker.isin(['ZION']),['IDRSSD','xName','xCert','Ticker','Parent','ParRSSD']] # current bank

IDRSSD                     xName   xCert Ticker Parent    ParRSSD
78  276579  ZIONS BANCORPORATION, NA  2270.0   ZION   ZION  1027004.0

In [78]:
rk5c.loc[rk5c.Ticker.isin(['ZION']),['IDRSSD','xName','xCert','Ticker','Parent','ParRSSD']] #historic banks merged in Zions 

IDRSSD                                 xName    xCert Ticker Parent  \
78     276579              ZIONS BANCORPORATION, NA   2270.0   ZION   ZION   
157    837260               CALIFORNIA BANK & TRUST  20852.0   ZION   ZION   
164    676656                        AMEGY BANK, NA  24107.0   ZION   ZION   
314   1004368              NATIONAL BANK OF ARIZONA  20626.0   ZION   ZION   
338    456960                     NEVADA STATE BANK  18113.0   ZION   ZION   
446    933957              VECTRA BANK COLORADO, NA   2993.0   ZION   ZION   
726    807665                     VINEYARD BANK, NA  23556.0   ZION   ZION   
1120   458263                  STOCKMEN'S BANK, THE  23182.0   ZION   ZION   
1170  1198641  COMMERCE BANK OF WASHINGTON, NA, THE  27298.0   ZION   ZION   
1233  2479310                     SILVER STATE BANK  34194.0   ZION   ZION   
1665   720465                         ALLIANCE BANK  23124.0   ZION   ZION   
1736   618656                            KLEIN BANK  22866.0   ZION   ZION   
2769   105950          MINNEQUA BANK OF PUEBLO, THE   1773.0   ZION   ZION   
3250   576158                             MAXIMBANK   9017.0   ZION   ZION   
4160   309664                           RANCHO BANK  22934.0   ZION   ZION   
4255   521550                        LONE STAR BANK  22304.0   ZION   ZION   
4736  2068134            GREAT BASIN BANK OF NEVADA  33824.0   ZION   ZION   
6268   626754              LEAGUE CITY BANK & TRUST  17577.0   ZION   ZION   
6460  3161993                           CHOICE BANK  57469.0   ZION   ZION   
6745   687269        INTERCONTINENTAL NATIONAL BANK  19112.0   ZION   ZION   
6780   456670                   FRONTIER STATE BANK  24227.0   ZION   ZION   
8459  3374186          COMMERCE BANK OF OREGON, THE  58223.0   ZION   ZION   
9922  2907448               SOUTHLAND BUSINESS BANK  35287.0   ZION   ZION   

        ParRSSD  
78    1027004.0  
157   1027004.0  
164   1027004.0  
314   1027004.0  
338   1027004.0  
446   1027004.0  
726   1027004.0  
1120  1027004.0  
1170  1027004.0  
1233  1027004.0  
1665  1027004.0  
1736  1027004.0  
2769  1027004.0  
3250  1027004.0  
4160  1027004.0  
4255  1027004.0  
4736  1027004.0  
6268  1027004.0  
6460  1027004.0  
6745  1027004.0  
6780  1027004.0  
8459  1027004.0  
9922  1027004.0